![](exercise_5_8_1.png)
![](exercise_5_8_2.png)
![](off_policy_mc_control.png)

In [48]:
def look_up(dict_):
    def look_up_(x):
        return dict_[x]
    return look_up_

z = look_up({1:2, 2: 3})


In [ ]:
def off_policy_mc_control():
    
    action_value = {}
    normalizing_constant = {}

    count = 1

    while True:
        
        behaviour_policy = epsilon_greedy_policy(target_policy, EPSILON)
        episode = run_episode(behaviour_policy, state_0)
        episode = list(reversed(x))
        
        G = 0
        W = 1
        
        # Iterate in tuples (t + 1, t)
        for new_state, new_action, new_reward, state, action, reward in zip(episode, episode[1:]):
            
            G = GAMMA * G + new_reward
            

            normalizing_constant[(state, action)] = normalizing_constant.get((state, action), 0) + W
            
            action_value[(state, action)] = (
                action_value.get((state, action), 0) +
                W / normalizing_constant.get((state, action), 0) * (G - action_value.get((state, action), 0))
            )
            
            target_policy[state] = best_action(state, action_value)
            
            if action != target_policy[state]:
                break
                
            W = W / behaviour_policy(state, action)


def run_episode(behaviour_policy, state_0):
    
    state = state_0.copy()
    episode = [(state, 0, -1)]
    
    while cell_type(state) != 'GOAL':
        
        action = behaviour_policy(state)
        
        state = move_car(state, action)
        
        episode.append(
            (state, action, -1)
        )
    
    return episode


def epsilon_greedy_policy(target_policy, epsilon = 0.1):
    def policy(state):
        if random.random() <= epsilon:
            return random.choice(actions(state))
        return target_policy[state]
    return policy

0.678572456155053

# Implementation

In [28]:
import pandas as pd
import numpy as np
from itertools import product
import random


MAP = pd.read_csv('maps/map1.csv', header=None).values
CELL_TYPE_MAP = {
    0: 'WALL',
    1: 'TRACK',
    2: 'GOAL',
    3: 'START',
}

EPSILON = 0.1

def states():
    """Generates all possible states.
    """
    for x, y in product(range(MAP.shape[0]), range(MAP.shape[1])):
        yield (x, y)
        
def cell_type(x, y):
    return CELL_TYPE_MAP[MAP[x, y]]


In [36]:
def actions(state):
    """Generate all velocity vectors at a given state.
    """
    _, _, dx, dy = state
    
    for ddx in [-1, 0, 1]:
        for ddy in [-1, 0, 1]:
            
            new_dx = dx + ddx
            new_dy = dy + ddy
            
            
            # Both velocity components are restricted to be positive and less than 5.
            if not (0 <= new_dx <= 5 and 0 <= new_dy <= 5):
                continue
            
            yield (new_dx, new_dy)
            

def move_car(state, action):
    """Move a car using the new velocity vector.
    """
    x, y, dx, dy = state
    ddx, ddy = action
    
    new_dx = dx + ddx
    new_dy = dy + ddy
    
    new_x = x + new_dx
    new_y = y + new_dy
    
    return (new_x, new_y, new_dx, new_dy)
            
assert len(list(actions((0, 0, 1, 1)))) == 9, 'total of 9 actions.'

[[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
 [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 1, 1, 1, 1, 1, 1, 1, 0,